# Linear Regression Real Estate

### Import required libraries

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
import pyspark

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [ ]:
spark = SparkSession\
        .builder\
        .appName("Real Estate")\
        .getOrCreate()
sc = spark.sparkContext

### Load the dataset from sklearn datasets

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
print(boston.keys())

<font color='green'>
    <ol>
        <li>data - contains the features </li>
        <li>target - contains the dependent variable </li>
        <li>feature_names - column headings </li>
        <li>DESCR - description of the dataset </li>
        <li>filename - dataset file name </li>
    </ol>
</font>

In [ ]:
print(boston.data.shape)

### Print the DESCR attribute in the data

In [ ]:
print(boston.DESCR)

### Convert the features (independent variables) into data frame

In [ ]:
bos = pd.DataFrame(boston.data)
bos.head()

# Notice the header columns are not populated

#### Notice the feature names are not set. Set them on the data frame

In [ ]:
bos.columns = boston.feature_names
bos.head()

### Print the shape of the target variable

In [ ]:
boston.target.shape

### Add the target column to the dataset and display first 5 rows/samples

In [ ]:
# add the price to the data frame
bos['PRICE'] = boston.target
bos.head()

### Display the last 10 rows/samples

In [ ]:
bos.tail(10)

### Display the std deviation, mean, min, max, etc of the dataset

In [ ]:
# Get the data description e.g. count, mean, standard deviation, etc.
pd.set_option('precision', 3)
bos.describe()

### Display the data types of features and target

In [ ]:
# display the data types
bos.info()

### Create a scatter plot of the Price v/s number of rooms

In [ ]:
plt.scatter(x = bos['PRICE'], y = bos['RM'])
plt.xlabel('Price in $1,000')
plt.ylabel('Number of Rooms')
plt.title('Rooms to Price')
display()

### Display all features "null" count

In [ ]:
# check if there are any null values in our features
bos.isnull().sum()

##### Looks like our data set is clean - no nulls

### Create Spark Data Frame from bos (pandas data frame)
Display the columns

In [ ]:
df1 = spark.createDataFrame(bos)

In [ ]:
df1.columns

In [ ]:
cols = df1.columns
cols = cols[:-1]
print(cols)

<font color = 'red'>
<h1>Spark Processing</h1>
</font>

<font color = 'red'>
<h3> Create Features and Target</h3>
1. Using Vector Assembler create features<br>
2. Transform features<br>
3. print the schema<br>
4. Display the first row<br>
</font>

In [ ]:
features = VectorAssembler(inputCols = df1.columns, outputCol = 'features')

In [ ]:
data = features.transform(df1)

In [ ]:
data.printSchema()

In [ ]:
data.head(1)

<font color = 'red'>
    <h3>Apply Standard Scaler to the features</h3>    
</font>

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="stdFeatures",
                            withStd=True, withMean=False)

In [ ]:
scalerModel = scaler.fit(data)

In [ ]:
scaledData = scalerModel.transform(data)

In [ ]:
pd.set_option('precision', 3)
scaledData.select('stdFeatures', 'features').toPandas().head()

<font color = 'red'>
<h3> Add the target variable PRICE</h3>
1. Create new variable that will hold both standard features and PRICE<br>
2. Print the schema of this new variable<br>
3. Display the first row of the new variable<br>
</font>

In [ ]:
data_matrix = scaledData.select('stdFeatures', 'PRICE')

In [ ]:
data_matrix.printSchema()

In [ ]:
data_matrix.head(1)

<font color = 'red'>
    <h3>Create Training and Test sets</h3>
</font>

In [ ]:
train, test = data_matrix.randomSplit([0.7, 0.3])

<font color = 'red'>
    <h3>Linear Regression</h3>
    <ol>
        <li>Create Linear Regressor Instance</li>
        <li>Train the algorithm on training data to create the model</li>
        <li>Make predictions with the model</li>
    </ol>
</font>

In [ ]:
# Define LinearRegression algorithm
lr = LinearRegression(labelCol = 'PRICE', featuresCol = 'stdFeatures', maxIter = 10, regParam = 0.3)

In [ ]:
model = lr.fit(train)

In [ ]:
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

<font color='green'>***R-squared is a statistical measure of how close the data are to the fitted regression line. It is also known as the coefficient of determination, or the coefficient of multiple determination for multiple regression.<br> R-Squared values are useful in showing how good a line fits. These values are typically between 0 and 1*** </font>

In [ ]:
trainingSummary = model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

In [ ]:
train.describe().show()

In [ ]:
pd.set_option('precision', 1)
predictions = model.transform(test)
predictions.select("prediction","PRICE","stdFeatures").toPandas().head(20)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="PRICE",metricName="r2")
print("R Squared (R2) on test data = %g" % evaluator.evaluate(predictions))

In [ ]:
test_result = model.evaluate(test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)